In [1]:
include("./general_utils/integrators.jl")
include("./general_utils/potentials.jl")
include("./general_utils/diffusion_tensors.jl")
include("./general_utils/probability_utils.jl")
include("./experiment_utils/experiments1D.jl")
include("./general_utils/integrator_utils.jl")
include("./general_utils/calculus.jl")

Main.Calculus

In [2]:
import .Integrators: eugen_gilles1D, euler_maruyama1D, hummer_leimkuhler_matthews1D, leimkuhler_matthews1D, leimkuhler_matthews_markovian1D, limit_method_with_variable_diffusion1D
import .Potentials: softWell1D, q4Potential
import .DiffusionTensors: Dconst1D, Dabs1D, Dquadratic1D, Dcosperturb1D
import .ProbabilityUtils: compute_1D_invariant_distribution
import .Experiments: master_1D_experiment
import .IntegratorUtils: MT2_1D, W2Ito1_1D
import .Calculus: differentiate1D

In [3]:
exp_name = "eugen_gilles_cos_LMVD_1D_100K"    # Name
master_dir = "simulation_results"  # Directory to save results in
T = 100000                           # length of simulation
sigma = 1                            # value of kT (noise amplitude scaling)
num_repeats = 6     

# The potential and diffusion coefficents to use
potential = q4Potential
diffusion = Dcosperturb1D

save_dir = "$(master_dir)/$(exp_name)"
chunk_size = 100000;

In [4]:
# The step sizes to use (to use a single step size, set stepsizes = [stepsize])
num_step_sizes = 10

# The range of stepsizes to use (defualt 10 step sizes in the range 10^{-2} to 10^{-1})
stepsizes = 10 .^ range(-2.0,stop=-1.0,length=num_step_sizes);

In [5]:
# The integrators to use (comma separated list)
integrators = [limit_method_with_variable_diffusion1D, eugen_gilles1D];

In [6]:
# The histogram parameters for binning
xmin = -5
xmax = 5
n_bins = 30

bin_boundaries = range(xmin, xmax, length=n_bins+1);

In [7]:
noise_integrator = W2Ito1_1D
n = 1

1

In [8]:
@info "Running: $(exp_name)"

master_1D_experiment(integrators, num_repeats, potential, diffusion, T, sigma, stepsizes, bin_boundaries, save_dir; chunk_size=chunk_size, x0=nothing, noise_integrator=noise_integrator, n=n)

[ Info: Running: eugen_gilles_cos_LMVD_1D_100K
[ Info: Computing the Invariant Distribution


Integral of f: 2.1558005495409276
Error in integral of f: 7.529340034491103e-10


[ Info: Running Experiments
[ Info: Running limit_method_with_variable_diffusion1D experiment
[ Info: Created directory simulation_results/eugen_gilles_cos_LMVD_1D_100K
[ Info: Saving metadata
16.7%┣███████▏                                   ┫ 1/6 [00:01<Inf:Inf, InfGs/it]
100.0%┣███████████████████████████████████████████████┫ 6/6 [00:01<00:00, 4it/s]
[ Info: Saving data
[ Info: Mean L1 errors: [0.00016039478140620632; 0.0001870047074835813; 0.00019724911178784692; 0.00021083197856729268; 0.00019109941246530453; 0.00024242948954306175; 0.0002519750946318988; 0.0003000728356273731; 0.00039360453562445314; 0.0006814981184621463;;]
[ Info: Standard deviation of L1 errors: [4.773105790615298e-5; 4.720894901841898e-5; 4.244027974936376e-5; 3.478855817427656e-5; 2.944369995027645e-5; 4.095142029895217e-5; 5.167050002324459e-5; 6.548536022609307e-5; 5.6602937599093957e-5; 4.256051442384715e-5;;]
[ Info: Running eugen_gilles1D experiment
16.7%┣███████▏                                   ┫ 1/6 

In [5]:
using Plots
using Random

x0 = 0
Vprime = differentiate1D(potential)
D_squared = x -> diffusion(x)^2
div_DDT = differentiate1D(D_squared)
sigma = 1
m = 10000
dt = 0.01
# reset random seed for reproducibility
Random.seed!(1234)
x_MT_2_1D, _ = eugen_gilles1D(x0, Vprime, diffusion, div_DDT, sigma::Number, m::Integer, dt::Number, nothing, MT2_1D);
Random.seed!(1234)
x_W2Ito1_1D, _ = eugen_gilles1D(x0, Vprime, diffusion, div_DDT, sigma::Number, m::Integer, dt::Number, nothing, W2Ito1_1D);
Random.seed!(1234)
div_D = differentiate1D(diffusion)
x_LMVD, _ = limit_method_with_variable_diffusion1D(x0, Vprime, diffusion, div_D, sigma::Number, m::Integer, dt::Number, nothing);

plot(x_MT_2_1D, label="MT_2")
#plot!(x_W2Ito1_1D, label="W2Ito1")
display(plot!(x_LMVD, label="LMVD"))

#display(plot(x_MT_2_1D - x_W2Ito1_1D, label="MT_2 - W2Ito1"))
#display(plot(x_MT_2_1D - x_EM, label="MT_2 - EM"))

LoadError: UndefVarError: `Dprime` not defined

In [20]:
using DifferentialEquations

α = 1
β = 1
u₀ = 1 / 2
f(u, p, t) = 0
g(u, p, t) = β * u
dt = 0.0001
tspan = (0.0, 10000.0)
prob = SDEProblem(f, g, u₀, (0.0, 10000.0))

sol = solve(prob, TangXiaoSROCK2(), dt = dt)
using Plots
plot(sol)

LoadError: InexactError: Int64(NaN)

# TODO List

<!-- 1. Make simplified function for running 1D experiments without transforms [Done]
2. Make simplified function for running 2D experiments without transforms
3. Make simplified function for running ND experiments without transforms
4. Implement 2D version of LMVD
5. Implement ND version of LMVD
6. Implement 1D version of Eugen-Gilles method [Done]
7. Implement 2D version of Eugen-Gilles method
8. Implement ND version of Eugen-Gilles method
9. Improved automated plot creation (with multiple integrators per figure)
10. Improved data management (create .txt files with mean and std convergence lines for each integrator)
11. Plots of compute time for fixed error
12. Comparison of effective stepsize
13. More interesting 2D problem where tensorial noise could be beneficial
14. Statistics problem? -->

1. Replace sqrt(dt) with one evaluation for efficiency
2. Fix implementation of 1D LMVD to work with new notation
3. Run long experiments comparing performance of LMVD, EG, EM and HLM methods on 1D problem. Save plots and data
4. Implement 2D version of Eugen-Gilles method